# 1. Инициализация датасета

In [1]:
from src.data_utils import create_dataset, LSTMDataset

In [2]:
train, val, test = create_dataset()


	Create dataset



100%|██████████| 1600498/1600498 [00:18<00:00, 84980.45it/s]


	Shuffle
	Split


In [3]:
from transformers import AutoTokenizer
name_tokenizer='bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(name_tokenizer)

In [4]:
train_dataset = LSTMDataset(train, tokenizer, seq_len=30)
val_dataset = LSTMDataset(val, tokenizer, seq_len=30)
test_dataset = LSTMDataset(test, tokenizer, seq_len=30)

In [5]:
from torch.utils.data import DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence


In [6]:
def collate_fn(batch):
    texts = [item['input'] for item in batch]
    labels =  torch.tensor([item['target'] for item in batch], dtype=torch.long)
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0) 


    return {
        'input': padded_texts, 
        'target': labels
    }

In [12]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)


# 2. Создание и обучение LSTM

In [8]:
from src.lstm_model import MyLSTM
from src.lstm_train import train as train_lstm
from src.eval_lstm import eval as eval_lstm

In [9]:

vocab_size = tokenizer.vocab_size  
hidden_dim = 128
model = MyLSTM(vocab_size, hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [10]:
torch.cuda.is_available()

True

In [ ]:
n_epoch = 6
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
train_lstm(
    model,
    train_loader,
    optimizer,
    tokenizer,
    val_loader,
    criterion,
    n_epoch
)

  0%|          | 0/63098 [00:00<?, ?it/s]

  0%|          | 267/63098 [00:40<2:05:40,  8.33it/s]

# 3. Использование трансформера

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, collate_fn=collate_fn)